<div style="position: absolute; top: 0; left: 0; font-family: 'Garamond'; font-size: 16px;">
    <a href="https://github.com/patriciaapenat" style="text-decoration: none; color: inherit;">Patricia Peña Torres</a>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 48px;">
    <strong>Proyecto final, BRFSS-clustering</strong>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 36px;">
    <strong>0.2. Análisis exploratorio</strong>
</div>

__________________

<div style="font-family: 'Garamond'; font-size: 14px;">

En este notebook se llava a cabo lo relativo al análisis exploratorio, por la naturaleza de los datos este EDA se ha centrado principalmente en variables demográficas
    
</div>

<div style="font-family: 'Garamond'; font-size: 16px;">
    <strong>Configuración del entorno de trabajo</strong>
</div>

In [1]:
import pandas as pd
import findspark
findspark.init()
import pyspark
import random
import os.path
import seaborn as sns
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from pyspark.sql import DataFrame
import pickle
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.ml.feature import Imputer
from functools import reduce
from pyspark.sql import DataFrame
import warnings
from pyspark.ml import Pipeline

# Ignorar advertencias deprecated
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# configurar gráficos
sns.set(style="whitegrid", context="notebook", palette="mako")

<div style="font-family: 'Garamond'; font-size: 14px;">
    <strong>Configuración de Spark</strong>
</div>

In [3]:
# Si hay un SparkContext existente, debemos cerrarlo antes de crear uno nuevo
if 'sc' in locals() and sc:
    sc.stop()  # Detener el SparkContext anterior si existe

# Configuración de Spark
conf = (
    SparkConf()
    .setAppName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .setMaster("local[1]")  # Modo local con un hilo para ejecución
    .set("spark.driver.host", "127.0.0.1")  # Dirección del host del driver
    .set("spark.executor.heartbeatInterval", "3600s")  # Intervalo de latido del executor
    .set("spark.network.timeout", "7200s")  # Tiempo de espera de la red
    .set("spark.executor.memory", "8g")  # Memoria asignada para cada executor
    .set("spark.driver.memory", "8g")  # Memoria asignada para el driver
)

# Crear un nuevo SparkContext con la configuración especificada
sc = SparkContext(conf=conf)

# Configuración de SparkSession (interfaz de alto nivel para trabajar con datos estructurados en Spark)
spark = (
    SparkSession.builder
    .appName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .config("spark.sql.repl.eagerEval.enabled", True)  # Habilitar la evaluación perezosa en Spark SQL REPL
    .config("spark.sql.repl.eagerEval.maxNumRows", 1000)  # Número máximo de filas a mostrar en la evaluación perezosa
    .getOrCreate()  # Obtener la sesión Spark existente o crear una nueva si no existe
) 

<div style="font-family: 'Garamond'; font-size: 14px;">
    <strong>Lectura del archivo</strong>
</div>

In [4]:
df = spark.read.format("csv").option("header", "true").load(r"C:\Users\patri\OneDrive - UAB\Documentos\GitHub\BRFSS-clustering\datos\BRFSS_Cleaner_2022.csv")

In [5]:
# Imprimir las dimensiones del DataFrame
print(f"Dimensiones del DataFrame: Filas={df.count()}, Columnas={len(df.columns)}")

Dimensiones del DataFrame: Filas=363130, Columnas=211



# Imputación de valores nulos

Es crucial abordar la presencia de valores nulos en nuestros datos, ya que estos podrían generar errores al intentar entrenar algoritmos. Ante esta situación, contamos con dos opciones principales:

1. **Imputación Estadística:**
   Podemos optar por realizar una imputación estadística, donde rellenamos los valores nulos con estadísticas descriptivas, como la media. Esta técnica es útil cuando buscamos proporcionar valores representativos para los datos faltantes.

2. **Relleno con Valor Específico:**
   Otra opción es utilizar el método `fillna` para rellenar los valores nulos con un valor específico, como 0. En este contexto, asignamos el valor 0 para indicar que cierta información "no aplica".

En nuestra estrategia, hemos aplicado métodos diferentes según la naturaleza de las variables. Algunas variables, relacionadas con preguntas que podrían no aplicar a todos los casos, se imputaron con el método de relleno con un valor específico. Por otro lado, para aquellas variables donde los valores simplemente no estaban disponibles, optamos por la imputación estadística utilizando la media, buscando proporcionar una estimación razonable para completar los datos faltantes.

In [6]:
# Convertir todas las columnas a tipo numérico
for column_name in df.columns:
    df = df.withColumn(column_name, col(column_name).cast("double"))

In [7]:
# Crear el imputador
imputer = Imputer(
    inputCols=df.columns,
    outputCols=[f"{col}_imputed" for col in df.columns],
    missingValue=float('nan'),
    strategy='mean'
)

# Ajustar y transformar el DataFrame
model = imputer.fit(df)
df_imputed = model.transform(df)

In [8]:
cols_to_drop = [col_name for col_name in df_imputed.columns if "_imputed" not in col_name]

In [9]:
df_imputed2 = df_imputed.drop(*cols_to_drop)

In [10]:
# Guardar el DataFrame como CSV
df_imputed2.write.csv("C:\\Users\\patri\\OneDrive - UAB\\Documentos\\GitHub\\BRFSS-clustering\\datos\\BRFSS_imputated_2022.csv", header=True, mode="overwrite")

In [11]:
# Iterar sobre todas las columnas sin limitación
for i in range(0, len(columnas), tamaño_grupo):
    grupo_columnas = columnas[i:i + tamaño_grupo]

    # Seleccionar solo las columnas del grupo actual
    df_grupo = df_imputed2.select(grupo_columnas)

    # Filtrar los valores nulos en el DataFrame
    df_grupo = df_grupo.dropna()

    # Verificar si el DataFrame es vacío después de filtrar los nulos
    if df_grupo.count() == 0:
        print(f"El DataFrame df_grupo está vacío después de filtrar los nulos para las columnas: {grupo_columnas}")
        continue

    # Resto del código para la creación del boxplot
    plt.figure(figsize=(15, 8))
    sns.boxplot(data=df_grupo.toPandas(), palette='mako')
    plt.title(f'Boxplots para las columnas: {", ".join(grupo_columnas)}')
    plt.xticks(rotation=90)

    # Guardar el gráfico en la ruta especificada
    nombre_archivo = f"boxplot_grupo_{i + 1}_{min(i + tamaño_grupo, len(columnas))}.png"
    ruta_completa = ruta_guardado + nombre_archivo
    plt.savefig(ruta_completa)

    # Cerrar la figura para liberar memoria
    plt.close()

NameError: name 'columnas' is not defined